# Getting started

## Before you start

## Installs

In [1]:
# ! pip install promptolution

## Imports

In [2]:
from promptolution.optimizers import get_optimizer
from promptolution.llms import get_llm
from promptolution.predictors.classificator import Classificator
from promptolution.tasks import get_task
from promptolution.config import Config

c:\Users\tzehl\Documents\programming\promptolution\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## set up llms, predictor, tasks and optimizer

In [3]:
config = Config(
    task_name="custom",
    ds_path="../data_sets/sst-5/",
    n_steps=10,
    optimizer="evopromptde",
    meta_llm="meta-llama/Meta-Llama-3-8B-Instruct",
    evaluation_llm="meta-llama/Meta-Llama-3-8B-Instruct",
    downstream_llm="meta-llama/Meta-Llama-3-8B-Instruct",
    meta_prompt_path="../templates/evoprompt_de_template_task_desc.txt"
)

In [4]:
task = get_task(config)

In [5]:
token = open("../deepinfratoken.txt", "r").read()
llm = get_llm(config.meta_llm, token=token)

In [6]:
predictor = Classificator(llm, classes=task.classes)

In [7]:
optimizer = get_optimizer(
    config, 
    meta_llm=llm,
    prompt_template=config.meta_prompt_path,
    initial_prompts=task.initial_population, 
    task=task,
    predictor=predictor
)

In [8]:
optimizer.optimize(n_steps=config.n_steps)

RuntimeError: asyncio.run() cannot be called from a running event loop